In [2]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torch.utils.data import DataLoader
from torcheval.metrics import R2Score
import pandas as pd
import torch
from torch.utils.data import random_split
import shutil
import os
from dataset import PGLSDataset
from models import PGLSModel, SimpleTabularModel, EnsemblePGLSModel, PGLSModelLateFusion
import timm
from torchrec.models.deepfm import DeepFM, DenseArch
from torchmetrics.functional import r2_score

/home/piotr/.local/lib/python3.10/site-packages/fbgemm_gpu/fbgemm_gpu_py.so: undefined symbol: _ZNK3c105Error4whatEv
/home/piotr/.local/lib/python3.10/site-packages/fbgemm_gpu/experimental/gen_ai/fbgemm_gpu_experimental_gen_ai_py.so: cannot open shared object file: No such file or directory


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


In [4]:
for folder in ["data/train_images", "data/test_images"]:
    if "0" not in os.listdir(folder):
        print("Moving images to 0 folder")
        os.makedirs(f"{folder}/0")
        for filename in os.listdir(folder):
            if filename.lower().endswith(".jpeg"):
                source_path = os.path.join(folder, filename)
                target_path = os.path.join(f"{folder}/0", filename)

                shutil.move(source_path, target_path)

In [5]:
transform = transforms.Compose([
    transforms.Resize((288, 288)),  # Resize the image to 224x224
    transforms.ToTensor(),         # Convert PIL image to tensor (H x W x C) in the range [0.0, 1.0]
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])

batch_size = 16     # use 4 if problem with GPU memory

In [6]:
train_images_path = 'data/train_images'
test_images_path = 'data/test_images'

train_csv_path = 'data/train.csv'
test_csv_path = 'data/test.csv'


tabular_data = pd.read_csv(train_csv_path)
targets = ["X4", "X11", "X18", "X26", "X50", "X3112"]

# Filter data
upper_values = {}
for target in targets:
    upper_values[target] = tabular_data[target+"_mean"].quantile(0.99)
    tabular_data = tabular_data[tabular_data[target+"_mean"] < upper_values[target]]
    tabular_data = tabular_data[tabular_data[target+"_mean"] > 0]

# Normalize the targets
original_means = tabular_data[[f"{target}_mean" for target in targets]].mean()
original_stds = tabular_data[[f"{target}_mean" for target in targets]].std()
tabular_data[[f"{target}_mean" for target in targets]] = (tabular_data[[f"{target}_mean" for target in targets]] - original_means) / original_stds

# Normalize the features
tabular_input_size = 0
for column in tabular_data.columns:
    if column in ["id"]+[target+"_mean" for target in targets]+[target+"_sd" for target in targets]:
        continue
    tabular_input_size += 1
    min_val = tabular_data[column].min()
    max_val = tabular_data[column].max()
    tabular_data[column] = (tabular_data[column] - min_val) / (max_val - min_val)


test_tabular_data = pd.read_csv(test_csv_path)
# Normalize the features
for column in test_tabular_data.columns:
    if column in ["id"]:
        continue
    min_val = test_tabular_data[column].min()
    max_val = test_tabular_data[column].max()
    test_tabular_data[column] = (test_tabular_data[column] - min_val) / (max_val - min_val)

train_images_dataset = ImageFolder(root=train_images_path, transform=transform)
test_images_dataset = ImageFolder(root=test_images_path, transform=transform)

train_image_csv_dataset = PGLSDataset(tabular_data=tabular_data, image_folder=train_images_dataset, transform_csv=None)
train, val = random_split(train_image_csv_dataset, [int(0.8*len(train_image_csv_dataset)), len(train_image_csv_dataset) - int(0.8*len(train_image_csv_dataset))])
test_image_csv_dataset = PGLSDataset(tabular_data=test_tabular_data, image_folder=test_images_dataset, transform_csv=None)


train_data_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
val_data_loader = DataLoader(val, batch_size=batch_size, shuffle=True)
test_data_loader = DataLoader(test_image_csv_dataset, batch_size=batch_size, shuffle=False)

In [7]:
original_stds = torch.from_numpy(original_stds.values).float()
original_means = torch.from_numpy(original_means.values).float()
def denormalize_targets(targets):
    return targets * original_stds + original_means

In [8]:
BiT = timm.create_model(
    'resnetv2_101x1_bitm',
    pretrained=True,
    num_classes=0,
)

embedding_dim = tabular_input_size

tabular_model = DenseArch(in_features=tabular_input_size, hidden_layer_size=tabular_input_size * 2, embedding_dim=embedding_dim)

model = PGLSModelLateFusion(BiT, tabular_model, embedding_dim)
metric = R2Score()
criterion = r2_score
torch.cuda.empty_cache()

/home/piotr/.local/lib/python3.10/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name resnetv2_101x1_bitm to current resnetv2_101x1_bit.goog_in21k_ft_in1k.
  model = create_fn(


In [10]:
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
max_value_metrics_batch = float("-inf")
model_best_batch_path = "./models/BiT_best_batch.pt"
max_value_metrics_epoch = float("-inf")
model_best_epoch_path = "./models/BiT_best_epoch.pt"
model.train()
for epoch in range(2):
    print(f"Epoch {epoch}")
    for data in train_data_loader:
        image, features, targets = data
        image = image.to(device)
        features = features.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        outputs = model(image, features)
        loss = 1 - r2_score(outputs, targets)
        loss.backward()
        optimizer.step()
        outputs = outputs.to("cpu")
        targets = targets.to("cpu")
        outputs_denorm = denormalize_targets(outputs)
        targets_denorm = denormalize_targets(targets)
        metric.update(outputs_denorm, targets_denorm)
        print(loss.item())
        print(metric.compute())
        new_max_value_metrics_batch = metric.compute().item()

        if(new_max_value_metrics_batch > max_value_metrics_batch):
            max_value_metrics_batch = new_max_value_metrics_batch
            torch.save(model.state_dict(), model_best_batch_path)
    
    if(new_max_value_metrics_batch > max_value_metrics_epoch):
        max_value_metrics_epoch = new_max_value_metrics_batch
        torch.save(model.state_dict(), model_best_epoch_path)


Epoch 0
93.90036010742188
tensor(-89.1150)


KeyboardInterrupt: 

In [13]:
accumulated_loss = 0
iterations = 0
accumulated_r2 = 0
model.to(device)
model.eval()
with torch.no_grad():
    for data in val_data_loader:
        image, features, targets = data
        image = image.to(device)
        features = features.to(device)
        targets = targets.to(device)
        outputs = model(image, features)
        loss = criterion(outputs, targets)
        accumulated_loss += loss.item()
        iterations += 1
        outputs = outputs.to("cpu")
        targets = targets.to("cpu")
        outputs_denorm = denormalize_targets(outputs)
        targets_denorm = denormalize_targets(targets)
        metric.update(outputs_denorm, targets_denorm)
        accumulated_r2 += metric.compute().item()
print("Average loss on validation set: ", accumulated_loss/iterations)
print("Average R2 on validation set: ", accumulated_r2/iterations)

KeyboardInterrupt: 

In [9]:
predictions = []
model.eval()
model.to(device)
with torch.no_grad():
    for data in test_data_loader:
        image, features, targets = data
        image = image.to(device)
        features = features.to(device)
        targets = targets.to(device)
        outputs = model(image, features)
        outputs = outputs.to("cpu")
        outputs_denorm = denormalize_targets(outputs)
        predictions.append(outputs_denorm)
predictions = [item for sublist in predictions for item in sublist]

In [12]:
def save_predictions(predictions, test_csv_dataframe, filename):
    with open(filename, "w") as f:
        f.write("id,X4,X11,X18,X26,X50,X3112\n")
        for pred, id in zip(predictions, test_csv_dataframe["id"]):
            pred = [p.item() for p in pred]
            f.write(f"{id},{','.join([str(p) for p in pred])}\n")

In [38]:
save_predictions(predictions, test_tabular_data, "predictions.csv")

NameError: name 'save_predictions' is not defined